In [1]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

/home/earneodo/.conda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15796979847973171161, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11167681741
 locality {
   bus_id: 1
 }
 incarnation: 259531018230757504
 physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.8446 - acc: 0.3196 - val_loss: 1.5470 - val_acc: 0.4369
Epoch 2/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.5737 - acc: 0.4276 - val_loss: 1.3945 - val_acc: 0.4947
Epoch 3/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.4501 - acc: 0.4752 - val_loss: 1.2716 - val_acc: 0.5423
Epoch 4/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3599 - acc: 0.5118 - val_loss: 1.1840 - val_acc: 0.5808
Epoch 5/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.2831 - acc: 0.5429 - val_loss: 1.1138 - val_acc: 0.6066
Epoch 6/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.2248 - acc: 0.5648 - val_loss: 1.0723 - val_acc: 0.6184
Epoch 7/100
1563/1563 [==============================] - 17s 11ms/step

1563/1563 [==============================] - 16s 11ms/step - loss: 0.7558 - acc: 0.7447 - val_loss: 0.6534 - val_acc: 0.7848
Epoch 60/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7561 - acc: 0.7476 - val_loss: 0.6659 - val_acc: 0.7717
Epoch 61/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7540 - acc: 0.7457 - val_loss: 0.6191 - val_acc: 0.7929
Epoch 62/100
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7587 - acc: 0.7444 - val_loss: 0.6383 - val_acc: 0.7837
Epoch 63/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7548 - acc: 0.7454 - val_loss: 0.6445 - val_acc: 0.7967
Epoch 64/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7565 - acc: 0.7466 - val_loss: 0.6336 - val_acc: 0.7906
Epoch 65/100
1563/1563 [==============================] - 15s 10ms/step - loss: 0.7595 - acc: 0.7457 - val_loss: 0.6442 - val_acc: 0.7846
Epoch 66/100
1563/1563 [=======================

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.5'

In [ ]:
!conda upgrade -y keras